### Introduction

This notebook demonstrates how you can take advantage of [H2O4GPU](https://github.com/h2oai/h2o4gpu) and [Datatable](https://github.com/h2oai/datatable) to speed up model training.

It uses Kaggle Competation Data from [Predicting Molecular Properties](https://www.kaggle.com/c/champs-scalar-coupling)

A special weighting strategy is performed to be as close as possible to the actual challenge evaluation metric.

The score on a 5 fold CV is and LB score is 

Please see readme file for installation.

In [1]:
import numpy as np
import h2o4gpu as sklearn
from h2o4gpu.model_selection import GroupKFold
from h2o4gpu.metrics import mean_absolute_error
from h2o4gpu.preprocessing import LabelEncoder
import pandas as pd
import xgboost as xgb
from progressbar import progressbar

import datatable as dt
import gc

import matplotlib.pyplot as plt
%matplotlib inline

### Utility functions

In [2]:
def group_mean_log_mae(y_true, y_pred, mol_types, floor=1e-9):
    
    maes = []
    for aType in sorted(mol_types.unique()):
        #print(aType)
        the_mean = np.mean(np.abs(y_true.loc[mol_types==aType] - y_pred.loc[mol_types==aType]))
        #print(max(floor, the_mean))
        maes.append(max(floor, the_mean))
    print(np.mean(maes), np.log(maes).mean())
    return np.log(maes).mean()

### Read data

In [3]:
path = "../input/"
train = dt.fread(path + "train.csv")
test = dt.fread(path + "test.csv")

In [4]:
train.head(5)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,−11.257
2,2,dsgdb9nsd_000001,1,3,2JHH,−11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,−11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [5]:
structures = dt.fread(path + "structures.csv")
structures.shape

(2358657, 6)

In [6]:
structures[dt.f.molecule_name == "dsgdb9nsd_000002", :]

,molecule_name,atom_index,atom,x,y,z
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,dsgdb9nsd_000002,0,N,−0.0404261,1.02411,0.0625638
1,dsgdb9nsd_000002,1,H,0.0172575,0.0125452,−0.0273772
2,dsgdb9nsd_000002,2,H,0.915789,1.35875,−0.0287578
3,dsgdb9nsd_000002,3,H,−0.520278,1.34353,−0.775543


### Feature Engineering

#### Electronegativity and Radius

In [7]:
# Copied From https://www.kaggle.com/adrianoavelar/bond-calculaltion-lb-0-82
# Credits to https://www.kaggle.com/adrianoavelar

atomic_radius = {'H':0.38, 'C':0.77, 'N':0.75, 'O':0.73, 'F':0.71} # Without fudge factor
fudge_factor = 0.05
atomic_radius = {k:v + fudge_factor for k,v in atomic_radius.items()}
print("Atomatic Radius")
print(atomic_radius, flush=True)


electronegativity = {'H':2.2, 'C':2.55, 'N':3.04, 'O':3.44, 'F':3.98}
atoms = structures[:, 'atom'].to_numpy()[:, 0]
atoms_en = [electronegativity[x] for x in progressbar(atoms)]
atoms_rad = [atomic_radius[x] for x in progressbar(atoms)]

Atomatic Radius
{'H': 0.43, 'C': 0.8200000000000001, 'N': 0.8, 'O': 0.78, 'F': 0.76}


100% (2358657 of 2358657) |##############| Elapsed Time: 0:00:02 Time:  0:00:02
100% (2358657 of 2358657) |##############| Elapsed Time: 0:00:02 Time:  0:00:02


In [8]:
structures.cbind(dt.Frame({"EN": atoms_en, "rad": atoms_rad}))

#### Bond calculation

In [9]:
# Copied From https://www.kaggle.com/adrianoavelar/bond-calculaltion-lb-0-82
# Credits to https://www.kaggle.com/adrianoavelar

i_atom = structures[:, 'atom_index'].to_numpy()[:, 0]
p = structures[:, ['x', 'y', 'z']].to_numpy()
p_compare = p
m = structures[:, 'molecule_name'].to_numpy()[:, 0]
m_compare = m
r = structures[:, 'rad'].to_numpy()[:, 0]
r_compare = r

source_row = np.arange(structures.shape[0])
max_atoms = 28

bonds = np.zeros((structures.shape[0]+1, max_atoms+1), dtype=np.int8)
bond_dists = np.zeros((structures.shape[0]+1, max_atoms+1), dtype=np.float32)

print('Calculating the bonds', flush=True)

for i in progressbar(range(max_atoms-1)):
    p_compare = np.roll(p_compare, -1, axis=0)
    m_compare = np.roll(m_compare, -1, axis=0)
    r_compare = np.roll(r_compare, -1, axis=0)
    
    mask = np.where(m == m_compare, 1, 0) #Are we still comparing atoms in the same molecule?
    dists = np.linalg.norm(p - p_compare, axis=1) * mask
    r_bond = r + r_compare
    
    bond = np.where(np.logical_and(dists > 0.0001, dists < r_bond), 1, 0)
    
    source_row = source_row
    target_row = source_row + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_row = np.where(np.logical_or(target_row > structures.shape[0], mask==0), structures.shape[0], target_row) #If invalid target, write to dummy row
    
    source_atom = i_atom
    target_atom = i_atom + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_atom = np.where(np.logical_or(target_atom > max_atoms, mask==0), max_atoms, target_atom) #If invalid target, write to dummy col
    
    bonds[(source_row, target_atom)] = bond
    bonds[(target_row, source_atom)] = bond
    bond_dists[(source_row, target_atom)] = dists
    bond_dists[(target_row, source_atom)] = dists

bonds = np.delete(bonds, axis=0, obj=-1) #Delete dummy row
bonds = np.delete(bonds, axis=1, obj=-1) #Delete dummy col
bond_dists = np.delete(bond_dists, axis=0, obj=-1) #Delete dummy row
bond_dists = np.delete(bond_dists, axis=1, obj=-1) #Delete dummy col

print('Counting and condensing bonds', flush=True)

bonds_numeric = [[i for i,x in enumerate(row) if x] for row in progressbar(bonds)]
bond_lengths = [[dist for i,dist in enumerate(row) if i in bonds_numeric[j]] for j,row in enumerate(progressbar(bond_dists))]
# Note that datatable does not support numpy types, so need to cast to python float
bond_lengths_mean = [float(np.mean(x)) for x in progressbar(bond_lengths)]
n_bonds = [len(x) for x in progressbar(bonds_numeric)]

Calculating the bonds


100% (27 of 27) |########################| Elapsed Time: 0:00:06 Time:  0:00:06


Counting and condensing bonds


100% (2358657 of 2358657) |##############| Elapsed Time: 0:00:08 Time:  0:00:08
100% (2358657 of 2358657) |##############| Elapsed Time: 0:00:10 Time:  0:00:10
100% (2358657 of 2358657) |##############| Elapsed Time: 0:00:24 Time:  0:00:24
100% (2358657 of 2358657) |##############| Elapsed Time: 0:00:02 Time:  0:00:02


In [10]:
structures.cbind(dt.Frame({'n_bonds':n_bonds, 'bond_lengths_mean': bond_lengths_mean}))
structures.head()

,molecule_name,atom_index,atom,x,y,z,EN,rad,n_bonds,bond_lengths_mean
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪,▪▪▪▪▪▪▪▪
0,dsgdb9nsd_000001,0,C,−0.0126981,1.0858,0.008001,2.55,0.82,4,1.09195
1,dsgdb9nsd_000001,1,H,0.00215042,−0.00603132,0.00197612,2.2,0.43,1,1.09195
2,dsgdb9nsd_000001,2,H,1.01173,1.46375,0.000276575,2.2,0.43,1,1.09195
3,dsgdb9nsd_000001,3,H,−0.540815,1.44753,−0.876644,2.2,0.43,1,1.09195
4,dsgdb9nsd_000001,4,H,−0.523814,1.43793,0.906397,2.2,0.43,1,1.09195
5,dsgdb9nsd_000002,0,N,−0.0404261,1.02411,0.0625638,3.04,0.8,3,1.01719
6,dsgdb9nsd_000002,1,H,0.0172575,0.0125452,−0.0273772,2.2,0.43,1,1.01719
7,dsgdb9nsd_000002,2,H,0.915789,1.35875,−0.0287578,2.2,0.43,1,1.01719
8,dsgdb9nsd_000002,3,H,−0.520278,1.34353,−0.775543,2.2,0.43,1,1.01721


### Merge with train and test

In [11]:
col_names = structures.names

structures.names = [_f + "_0" if _f != "molecule_name" else _f for _f in col_names]
structures.key = ["molecule_name", "atom_index_0"]
train = train[:, :, dt.join(structures)]
test = test[:, :, dt.join(structures)]

structures.names = [_f + "_1" if _f != "molecule_name" else _f for _f in col_names]
structures.key = ["molecule_name", "atom_index_1"]
train = train[:, :, dt.join(structures)]
test = test[:, :, dt.join(structures)]

train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,…,z_1,EN_1,rad_1,n_bonds_1,bond_lengths_mean_1
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪,▪▪▪▪▪▪▪▪
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.00215042,−0.00603132,0.00197612,…,0.008001,2.55,0.82,4,1.09195
1,1,dsgdb9nsd_000001,1,2,2JHH,−11.257,H,0.00215042,−0.00603132,0.00197612,…,0.000276575,2.2,0.43,1,1.09195
2,2,dsgdb9nsd_000001,1,3,2JHH,−11.2548,H,0.00215042,−0.00603132,0.00197612,…,−0.876644,2.2,0.43,1,1.09195
3,3,dsgdb9nsd_000001,1,4,2JHH,−11.2543,H,0.00215042,−0.00603132,0.00197612,…,0.906397,2.2,0.43,1,1.09195
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.01173,1.46375,0.000276575,…,0.008001,2.55,0.82,4,1.09195
5,5,dsgdb9nsd_000001,2,3,2JHH,−11.2541,H,1.01173,1.46375,0.000276575,…,−0.876644,2.2,0.43,1,1.09195
6,6,dsgdb9nsd_000001,2,4,2JHH,−11.2548,H,1.01173,1.46375,0.000276575,…,0.906397,2.2,0.43,1,1.09195
7,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,H,−0.540815,1.44753,−0.876644,…,0.008001,2.55,0.82,4,1.09195
8,8,dsgdb9nsd_000001,3,4,2JHH,−11.2543,H,−0.540815,1.44753,−0.876644,…,0.906397,2.2,0.43,1,1.09195


In [12]:
train[dt.f.molecule_name == "dsgdb9nsd_000002", :]

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,…,z_1,EN_1,rad_1,n_bonds_1,bond_lengths_mean_1
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪,▪▪▪▪▪▪▪▪
0,10,dsgdb9nsd_000002,1,0,1JHN,32.6889,H,0.0172575,0.0125452,−0.0273772,…,0.0625638,3.04,0.8,3,1.01719
1,11,dsgdb9nsd_000002,1,2,2JHH,−11.1866,H,0.0172575,0.0125452,−0.0273772,…,−0.0287578,2.2,0.43,1,1.01719
2,12,dsgdb9nsd_000002,1,3,2JHH,−11.1757,H,0.0172575,0.0125452,−0.0273772,…,−0.775543,2.2,0.43,1,1.01721
3,13,dsgdb9nsd_000002,2,0,1JHN,32.6891,H,0.915789,1.35875,−0.0287578,…,0.0625638,3.04,0.8,3,1.01719
4,14,dsgdb9nsd_000002,2,3,2JHH,−11.1758,H,0.915789,1.35875,−0.0287578,…,−0.775543,2.2,0.43,1,1.01721
5,15,dsgdb9nsd_000002,3,0,1JHN,32.6905,H,−0.520278,1.34353,−0.775543,…,0.0625638,3.04,0.8,3,1.01719


### How to compute value_counts ?

In [13]:
train[:, dt.count(), dt.by("atom_0")]

,atom_0,C0
,▪▪▪▪,▪▪▪▪
0,H,4658147


In [14]:
train[:, dt.count(), dt.by("atom_1")]

,atom_1,C0
,▪▪▪▪,▪▪▪▪
0,C,3360469
1,H,968647
2,N,329031


### Simple Feature Engineering

#### L1 distance

In [15]:
cols = ["x", "y", "z"]
train.cbind(dt.Frame(
    {
        "l1_distance": np.sum(
            np.abs(train[:, [f+"_0" for f in cols]].to_numpy() 
                   - train[:, [f+"_1" for f in cols]].to_numpy()), axis=1)
    }
))
            
test.cbind(dt.Frame(
    {
        "l1_distance": np.sum(
            np.abs(test[:, [f+"_0" for f in cols]].to_numpy() 
                   - test[:, [f+"_1" for f in cols]].to_numpy()), axis=1)
    }
))


#### L2 distance

In [16]:
cols = ["x", "y", "z"]
train.cbind(dt.Frame(
    {
        "l2_distance": np.sum(
            (train[:, [f+"_0" for f in cols]].to_numpy() 
             - train[:, [f+"_1" for f in cols]].to_numpy()) ** 2, axis=1)
    }
))
            
test.cbind(dt.Frame(
    {
        "l2_distance": np.sum(
            (test[:, [f+"_0" for f in cols]].to_numpy() 
             - test[:, [f+"_1" for f in cols]].to_numpy()) ** 2, axis=1)
    }
))

#### Number of links in the molecules

In [17]:
trn_nb_links = train[:, dt.count(), dt.by("molecule_name")]
trn_nb_links.names = ["molecule_name", "nb_links"]
trn_nb_links.key = "molecule_name"
train = train[:, :, dt.join(trn_nb_links)]

In [18]:
tst_nb_links = test[:, dt.count(), dt.by("molecule_name")]
tst_nb_links.names = ["molecule_name", "nb_links"]
tst_nb_links.key = "molecule_name"
test = test[:, :, dt.join(tst_nb_links)]

### Encoding Categorical features

In [19]:
features = [f for f in train.names if f not in ["scalar_coupling_constant", "id", "molecule_name", "type", "predictions", "atom_0"]]
np.array(features)

array(['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'EN_0',
       'rad_0', 'n_bonds_0', 'bond_lengths_mean_0', 'atom_1', 'x_1',
       'y_1', 'z_1', 'EN_1', 'rad_1', 'n_bonds_1', 'bond_lengths_mean_1',
       'l1_distance', 'l2_distance', 'nb_links'], dtype='<U19')

In [20]:
# One of the coolest thing about DataTable ;-)
# You can select features on their type like train[:, [int, float]]
cat_feats = [f for f in train[:, str].names if f not in ["molecule_name", "type", "atom_0"]]
cat_feats

['atom_1']

In [21]:
lbl_encoders = {}
for f in cat_feats:
    lbl_encoders[f] = pd.Index(dt.unique(train[:, f]).to_numpy()[:, 0])
    train.cbind(dt.Frame({f+"_lbl_enc": lbl_encoders[f].get_indexer(train[:, f].to_numpy()[:, 0])}))
    test.cbind(dt.Frame({f+"_lbl_enc": lbl_encoders[f].get_indexer(test[:, f].to_numpy()[:, 0])}))
    features.remove(f)
    features.append(f+"_lbl_enc")

### Build a general model

We use weights to optimize for average MAE over the molecule types

In [22]:
def group_mean_log_mae_dt(y_true, y_pred, mol_types, floor=1e-9):
    
    maes = []
    np_types = mol_types.to_numpy()[:, 0] 
    np_true = y_true.to_numpy()[:, 0]

    for aType in sorted(sorted(dt.unique(mol_types).to_list()[0])):
        the_mean = np.mean(np.abs(np_true[np_types==aType] - y_pred[np_types==aType]))
        maes.append(max(floor, the_mean))
        
    print("Mean of Type MAE : %.4f | Eval Metric : %.4f" % (np.mean(maes), np.log(maes).mean()))
    
    return np.log(maes).mean()

In [23]:
target = "scalar_coupling_constant"
oof_preds = np.zeros(train.shape[0])    
models = []

for trn_, val_ in GroupKFold(5).split(X=train[:, "type"].to_numpy(), y=None, groups=train[:, "molecule_name"].to_numpy()[:, 0]):
    trn_type_sizes = train[trn_, :][:, dt.count(), dt.by("type")]
    val_type_sizes = train[val_, :][:, dt.count(), dt.by("type")]
    
    trn_type_sizes[:, "C0"] = trn_type_sizes[:, trn_.shape[0] / (dt.f.C0 * trn_type_sizes.shape[0])]
    val_type_sizes[:, "C0"] = val_type_sizes[:, val_.shape[0] / (dt.f.C0 * val_type_sizes.shape[0])]
    
    trn_type_sizes.key = ["type"]
    trn_weights = train[trn_, "type"][:, :, dt.join(trn_type_sizes)][:, "C0"].to_numpy()[:,0]
    val_type_sizes.key = ["type"]
    val_weights = train[val_, "type"][:, :, dt.join(val_type_sizes)][:, "C0"].to_numpy()[:,0]
    
    model = xgb.XGBRegressor(
        tree_method="gpu_hist",
        objective="reg:squarederror",
        n_estimators=5000, 
        learning_rate=.5,
        max_depth=9,
        sub_sample=.8,
        colsample_bytree=.8,
        reg_alpha=10,
        reg_lambda=5,
        n_jobs=-1,
        eval_metric="mae"
    )
    
    # Need to materialize views before sending to XGBoost
    trn_X = train[trn_, :]
    trn_X.materialize()
    val_X = train[val_, :]
    val_X.materialize()
    
    model.fit(
        trn_X[:, features], trn_X[:, target], sample_weight=trn_weights,
        eval_set=[(val_X[:, features], val_X[:, target])],
        sample_weight_eval_set=[val_weights],
        early_stopping_rounds=100,
        verbose=500,
    )
    
    oof_preds[val_] = model.predict(val_X[:, features])
    
    models.append(model)
        
    print("MEAN LOG MAE SCORE : %.6f" 
          % group_mean_log_mae_dt(
            y_true=val_X[:, target], 
            y_pred=oof_preds[val_], 
            mol_types=val_X[:, "type"], 
            floor=1e-9
        ))



[0]	validation_0-mae:10.813
Will train until validation_0-mae hasn't improved in 100 rounds.
[500]	validation_0-mae:1.0762
[1000]	validation_0-mae:1.04392
[1500]	validation_0-mae:1.02914
[2000]	validation_0-mae:1.0203
[2500]	validation_0-mae:1.01421
[3000]	validation_0-mae:1.0102
[3500]	validation_0-mae:1.00676
[4000]	validation_0-mae:1.00429
[4500]	validation_0-mae:1.00196
[4999]	validation_0-mae:1.00024
Mean of Type MAE : 1.0002 | Eval Metric : -0.1025
MEAN LOG MAE SCORE : -0.102507
[0]	validation_0-mae:10.8014
Will train until validation_0-mae hasn't improved in 100 rounds.
[500]	validation_0-mae:1.07225
[1000]	validation_0-mae:1.04
[1500]	validation_0-mae:1.02485
[2000]	validation_0-mae:1.01514
[2500]	validation_0-mae:1.00973
[3000]	validation_0-mae:1.00507
[3500]	validation_0-mae:1.00157
[4000]	validation_0-mae:0.998908
[4500]	validation_0-mae:0.996846
[4999]	validation_0-mae:0.994942
Mean of Type MAE : 0.9949 | Eval Metric : -0.1072
MEAN LOG MAE SCORE : -0.107164
[0]	validation_0

In [28]:
print("MEAN LOG MAE SCORE : %.6f" 
  % group_mean_log_mae_dt(
    y_true=train[:, target], 
    y_pred=oof_preds, 
    mol_types=train[:, "type"], 
    floor=1e-9
))

Mean of Type MAE : 0.9974 | Eval Metric : -0.1048
MEAN LOG MAE SCORE : -0.104776


### Predict Test target

In [29]:
sub_preds = np.zeros(test.shape[0])
test.materialize()

for model in models:
    sub_preds += model.predict(test[:, features]) / len(models)

### Submit files to Kaggle

In [30]:
preds_dt = test[:, "id"].copy()
preds_dt.cbind(dt.Frame({"scalar_coupling_constant": sub_preds}))
preds_dt.head()

,id,scalar_coupling_constant
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,4658147,15.6552
1,4658148,192.717
2,4658149,0.168881
3,4658150,193.485
4,4658151,18.0486
5,4658152,91.3877
6,4658153,1.36821
7,4658154,−6.93436
8,4658155,−8.74346


In [31]:
preds_dt.to_csv("champs_scalar_counpling_h2o4gpu_datatable.csv")

In [32]:
!kaggle competitions submit -f champs_scalar_counpling_h2o4gpu_datatable.csv -m "predictions with h2o4gpu and datatable" champs-scalar-coupling

100%|██████████████████████████████████████| 64.0M/64.0M [00:06<00:00, 10.6MB/s]
Successfully submitted to Predicting Molecular Properties